In [1]:
import os
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
%matplotlib inline

In [2]:
labels = []

In [3]:
filename = None

for filename in range(1, 3065):
    with h5py.File('./brain_tumor_dataset/imageData/{}.mat'.format(filename), 'r') as f:
        label = f['cjdata']['label'][0][0] - 1
        labels.append(int(label))

In [4]:
for i in range(len(labels)):
    if(labels[i]==0):
        labels[i] = "glioma"
    elif(labels[i]==1):
        labels[i] = "meningioma"
    else:
        labels[i] = "pituitary"

In [5]:
def importing_data(path):
    sample = []
    for filename in os.listdir(path):
        if "jpg" in filename:
            sample.append(filename)
    return sample

In [6]:
path = './brain_tumor_dataset/bt_images/'
train = importing_data(path)

train_data = pd.DataFrame({'image':train, 'label': labels})

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data, test_size = 0.1, shuffle = True, random_state = 42)

In [8]:
IMG_SIZE = 64
size = (IMG_SIZE,IMG_SIZE)

In [9]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [10]:
inputs = Input(shape = (64, 64, 1))
conv1 =  Conv2D(32, (3,3), activation = 'leaky_relu', padding = 'same', strides=(1,1), input_shape = [64,64,1])(inputs)
pool1 = MaxPooling2D(2)(conv1)
conv2 = Conv2D(64, (3,3), activation = 'leaky_relu', padding = 'same', strides=(1,1))(pool1)
pool2 = MaxPooling2D(2)(conv2)
conv3 = Conv2D(64, (3,3), activation = 'leaky_relu', padding = 'same', strides=(1,1))(pool2)
pool3 = MaxPooling2D(2)(conv3)
conv4 = Conv2D(128,(3,3), activation = 'leaky_relu', padding = 'same', strides=(1,1))(pool3)
pool4 = MaxPooling2D(2)(conv4)
conv5 = Conv2D(256,(3,3), activation = 'leaky_relu', padding = 'same', strides=(1,1))(pool4)
pool5 = MaxPooling2D(2)(conv5)
flatten = Flatten()(pool5)
dense1 = Dense(32, activation = 'leaky_relu')(flatten)
dropout1 = Dropout(0.25)(dense1)
outputs = Dense(3, activation = 'softmax')(dropout1)
model = Model(inputs=inputs, outputs=outputs)


model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0     

In [11]:
datagen = ImageDataGenerator(rescale = 1./255)

train_set = datagen.flow_from_dataframe(X_train,
                                        directory = './brain_tumor_dataset/bt_images/',
                                        x_col = 'image',
                                        y_col = 'label',
                                        target_size = size,
                                        color_mode = 'grayscale',
                                        class_mode = 'categorical',
                                        batch_size = 1,
                                        shuffle = True,
                                        interpolation = 'bilinear')

val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

    

model.compile(optimizer="adamax",loss='categorical_crossentropy',metrics=['accuracy']) 
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint("./models/bayesian_leaky_relu_adamax.h5", save_best_only=True)
history = model.fit(train_set, validation_data = val_set, epochs= 30, callbacks=[checkpoint_cb, es])   

Found 2757 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
Epoch 1/30
2757/2757 [==============================] - 144s 37ms/step - loss: 0.7248 - accuracy: 0.6783 - val_loss: 0.5034 - val_accuracy: 0.7818
Epoch 2/30
2757/2757 [==============================] - 50s 18ms/step - loss: 0.5227 - accuracy: 0.7940 - val_loss: 0.5259 - val_accuracy: 0.7590
Epoch 3/30
2757/2757 [==============================] - 49s 18ms/step - loss: 0.4408 - accuracy: 0.8161 - val_loss: 0.3590 - val_accuracy: 0.8567
Epoch 4/30
2757/2757 [==============================] - 49s 18ms/step - loss: 0.3534 - accuracy: 0.8636 - val_loss: 0.3074 - val_accuracy: 0.9023
Epoch 5/30
2757/2757 [==============================] - 49s 18ms/step - loss: 0.2707 - accuracy: 0.8988 - val_loss: 0.3037 - val_accuracy: 0.8730
Epoch 6/30
2757/2757 [==============================] - 51s 18ms/step - loss: 0.2154 - accuracy: 0.9202 - val_loss: 0.2119 - val_accuracy: 0.9088
Ep

In [10]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = np.expand_dims(img,0)
    return img

In [11]:
val = X_val["image"].values.tolist()

In [12]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_leaky_relu_adamax.h5"

In [13]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "adamax", loss='categorical_crossentropy', metrics =['accuracy'])

In [14]:
def perturb(x):
    adv_img = orig.copy()

    pixs = np.array(np.split(x, len(x)/3)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2]
    adv_img[loc] = val
    adv_img = np.expand_dims(adv_img,0)
    return adv_img

In [15]:
def optimize(x):
    adv_img = perturb(x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    pred_orig = np.argmax(prob_orig)

    return prob[0][pred_orig]

In [16]:
pred_adv = 0
prob_adv = 0
ct = 0
def callback(x, convergence):
    global pred_adv, prob_adv, ct
    adv_img = perturb(x)


    prob = keras_model.predict(preprocess(adv_img[0]))

    pred_adv = np.argmax(prob)
    prob_adv = prob[0][pred_adv]
    if pred_adv != pred_orig:
        ct = ct+1
        return True

In [19]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)


    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

    if(pred_orig != pred_adv):
        cv2.imwrite('./bayesian_outputs_leaky_relu_adamax/'+filename, adv_img[0])

Prob [pituitary]: 0.983400 --> Prob[pituitary]: 0.716268
Prob [glioma]: 0.606796 --> Prob[meningoma]: 0.833452
Prob [pituitary]: 0.958467 --> Prob[meningoma]: 0.543376
Prob [pituitary]: 0.997834 --> Prob[pituitary]: 0.981863
Prob [meningoma]: 0.736500 --> Prob[pituitary]: 0.901546
Prob [glioma]: 0.994244 --> Prob[glioma]: 0.932235
Prob [meningoma]: 0.999996 --> Prob[meningoma]: 0.999989
Prob [meningoma]: 0.816144 --> Prob[pituitary]: 0.626464
Prob [glioma]: 0.993269 --> Prob[glioma]: 0.977848
Prob [pituitary]: 0.976441 --> Prob[pituitary]: 0.922623
Prob [meningoma]: 0.991162 --> Prob[meningoma]: 0.823997
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000
Prob [glioma]: 0.994165 --> Prob[glioma]: 0.923773
Prob [pituitary]: 0.996485 --> Prob[pituitary]: 0.978395
Prob [meningoma]: 0.665582 --> Prob[pituitary]: 0.769165
Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999998
Prob [meningoma]: 0.938324 --> Prob[meningoma]: 0.619520
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.00

Prob [pituitary]: 0.743862 --> Prob[glioma]: 0.372636
Prob [meningoma]: 0.985050 --> Prob[glioma]: 0.592568
Prob [pituitary]: 0.792804 --> Prob[meningoma]: 0.549678
Prob [meningoma]: 0.999993 --> Prob[meningoma]: 0.999986
Prob [meningoma]: 0.994139 --> Prob[meningoma]: 0.942238
Prob [pituitary]: 0.767573 --> Prob[meningoma]: 0.540770
Prob [pituitary]: 0.924524 --> Prob[pituitary]: 0.752421
Prob [pituitary]: 0.998171 --> Prob[pituitary]: 0.977096
Prob [meningoma]: 0.904945 --> Prob[pituitary]: 0.533232
Prob [glioma]: 0.976703 --> Prob[glioma]: 0.848114
Prob [meningoma]: 0.968012 --> Prob[glioma]: 0.548411
Prob [glioma]: 0.910357 --> Prob[meningoma]: 0.806295
Prob [pituitary]: 0.652545 --> Prob[meningoma]: 0.582497
Prob [meningoma]: 0.999918 --> Prob[meningoma]: 0.999452
Prob [glioma]: 0.842045 --> Prob[meningoma]: 0.526971
Prob [pituitary]: 0.955669 --> Prob[meningoma]: 0.593278
Prob [meningoma]: 0.812351 --> Prob[meningoma]: 0.537152
Prob [meningoma]: 0.980998 --> Prob[meningoma]: 0.79

Prob [meningoma]: 0.997954 --> Prob[meningoma]: 0.988044
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000
Prob [glioma]: 0.984969 --> Prob[glioma]: 0.749345
Prob [meningoma]: 0.999843 --> Prob[meningoma]: 0.999238
Prob [glioma]: 0.998974 --> Prob[glioma]: 0.993775
Prob [pituitary]: 0.959847 --> Prob[pituitary]: 0.800449
Prob [pituitary]: 0.995642 --> Prob[pituitary]: 0.955868
Prob [glioma]: 0.999480 --> Prob[glioma]: 0.997436
Prob [glioma]: 0.983663 --> Prob[glioma]: 0.546783
Prob [pituitary]: 0.994785 --> Prob[pituitary]: 0.939060
Prob [meningoma]: 0.985480 --> Prob[meningoma]: 0.835569
Prob [meningoma]: 0.961782 --> Prob[meningoma]: 0.826554


In [20]:
print(ct)

88


In [17]:
datagen = ImageDataGenerator(rescale = 1./255)
adv_set = datagen.flow_from_dataframe(X_val,
                                      directory = './bayesian_outputs_leaky_relu_adamax/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 88 validated image filenames belonging to 3 classes.


D:\Softwares\anaconda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 219 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [14]:
predict = keras_model.evaluate(adv_set)

88/88 [==============================] - 49s 31ms/step - loss: 0.9367 - accuracy: 0.2955


In [15]:
path = './bayesian_outputs_leaky_relu_adamax/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val_df = X_val[~X_val.image.isin(adv_df.image)]

In [19]:
val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 307 validated image filenames belonging to 3 classes.


In [17]:
keras_model.evaluate(val_set)

219/219 [==============================] - 5s 23ms/step - loss: 0.0493 - accuracy: 0.9863


[0.04933905974030495, 0.9863013625144958]

In [21]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/bayesian_leaky_relu_adamax_adversarial.h5", save_best_only=True)

history_adv = keras_model.fit(adv_set, validation_data = val_set, epochs= 50, batch_size = 1, callbacks=[es, checkpoint_cb])   

Epoch 1/50
88/88 [==============================] - 50s 114ms/step - loss: 0.8669 - accuracy: 0.8068 - val_loss: 0.2811 - val_accuracy: 0.8990
Epoch 2/50
88/88 [==============================] - 2s 18ms/step - loss: 0.2983 - accuracy: 0.8864 - val_loss: 0.2636 - val_accuracy: 0.9186
Epoch 3/50
88/88 [==============================] - 2s 18ms/step - loss: 0.1295 - accuracy: 0.9659 - val_loss: 0.2657 - val_accuracy: 0.9251
Epoch 4/50
88/88 [==============================] - 2s 18ms/step - loss: 0.0840 - accuracy: 0.9773 - val_loss: 0.2464 - val_accuracy: 0.9349
Epoch 5/50
88/88 [==============================] - 2s 20ms/step - loss: 0.0575 - accuracy: 0.9886 - val_loss: 0.1991 - val_accuracy: 0.9381
Epoch 6/50
88/88 [==============================] - 2s 20ms/step - loss: 0.0194 - accuracy: 1.0000 - val_loss: 0.2546 - val_accuracy: 0.9283
Epoch 7/50
88/88 [==============================] - 2s 19ms/step - loss: 0.0148 - accuracy: 1.0000 - val_loss: 0.2941 - val_accuracy: 0.9088
Restoring m

In [23]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_leaky_relu_adamax_adversarial.h5"
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "adamax", loss='categorical_crossentropy', metrics =['accuracy'])

In [24]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

Prob [pituitary]: 0.999741 --> Prob[pituitary]: 0.997403 
Prob [glioma]: 0.985864 --> Prob[meningoma]: 0.541829 
Prob [pituitary]: 0.999532 --> Prob[pituitary]: 0.993425 
Prob [pituitary]: 0.998978 --> Prob[pituitary]: 0.994018 
Prob [pituitary]: 0.752986 --> Prob[meningoma]: 0.780621 
Prob [glioma]: 0.999610 --> Prob[glioma]: 0.995730 
Prob [meningoma]: 0.999618 --> Prob[meningoma]: 0.997965 
Prob [pituitary]: 0.990100 --> Prob[pituitary]: 0.937658 
Prob [glioma]: 0.998006 --> Prob[glioma]: 0.995125 
Prob [pituitary]: 0.974241 --> Prob[pituitary]: 0.923887 
Prob [meningoma]: 0.592029 --> Prob[pituitary]: 0.527645 
Prob [meningoma]: 0.999990 --> Prob[meningoma]: 0.999982 
Prob [glioma]: 0.995791 --> Prob[glioma]: 0.934652 
Prob [pituitary]: 0.999959 --> Prob[pituitary]: 0.999694 
Prob [meningoma]: 0.992131 --> Prob[meningoma]: 0.968717 
Prob [meningoma]: 0.999873 --> Prob[meningoma]: 0.999679 
Prob [meningoma]: 0.866013 --> Prob[meningoma]: 0.713570 
Prob [meningoma]: 0.998829 --> Prob

Prob [meningoma]: 0.992015 --> Prob[meningoma]: 0.964315 
Prob [pituitary]: 0.999717 --> Prob[pituitary]: 0.995156 
Prob [meningoma]: 0.992483 --> Prob[meningoma]: 0.973401 
Prob [pituitary]: 0.975162 --> Prob[pituitary]: 0.910412 
Prob [meningoma]: 0.999892 --> Prob[meningoma]: 0.999660 
Prob [meningoma]: 0.895795 --> Prob[meningoma]: 0.626331 
Prob [pituitary]: 0.938927 --> Prob[pituitary]: 0.695395 
Prob [pituitary]: 0.999337 --> Prob[pituitary]: 0.997379 
Prob [pituitary]: 0.999974 --> Prob[pituitary]: 0.999956 
Prob [pituitary]: 0.995546 --> Prob[pituitary]: 0.953855 
Prob [glioma]: 0.995745 --> Prob[glioma]: 0.976751 
Prob [meningoma]: 0.992337 --> Prob[meningoma]: 0.970993 
Prob [glioma]: 0.999886 --> Prob[glioma]: 0.998575 
Prob [pituitary]: 0.985023 --> Prob[pituitary]: 0.894649 
Prob [meningoma]: 0.975037 --> Prob[meningoma]: 0.928328 
Prob [glioma]: 0.999953 --> Prob[glioma]: 0.999724 
Prob [pituitary]: 0.997873 --> Prob[pituitary]: 0.986592 
Prob [glioma]: 0.572277 --> Prob

Prob [meningoma]: 0.991749 --> Prob[meningoma]: 0.946764 
Prob [pituitary]: 0.988914 --> Prob[pituitary]: 0.851441 
Prob [meningoma]: 0.945064 --> Prob[meningoma]: 0.757655 
Prob [meningoma]: 0.999986 --> Prob[meningoma]: 0.999973 
Prob [glioma]: 0.992505 --> Prob[glioma]: 0.872314 
Prob [meningoma]: 0.826112 --> Prob[meningoma]: 0.587866 
Prob [glioma]: 1.000000 --> Prob[glioma]: 1.000000 
Prob [pituitary]: 0.991677 --> Prob[pituitary]: 0.939766 
Prob [pituitary]: 0.999977 --> Prob[pituitary]: 0.999940 
Prob [glioma]: 0.999991 --> Prob[glioma]: 0.999983 
Prob [glioma]: 0.999522 --> Prob[glioma]: 0.993730 
Prob [pituitary]: 0.999897 --> Prob[pituitary]: 0.999431 
Prob [meningoma]: 0.846195 --> Prob[meningoma]: 0.729431 
Prob [meningoma]: 0.842630 --> Prob[meningoma]: 0.575078 


In [25]:
print(ct)

29
